In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(3)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
566,tt0230838,40000000,65754228,Sweet November,Keanu Reeves|Charlize Theron|Jason Isaacs|Greg...,Pat O'Connor,She Just Needed A Month To Change His Life For...,Nelson is a man devoted to his advertising car...,119,Drama|Romance,Bel Air Entertainment|Warner Bros.,2/16/2001,6.5,2001
1464,tt0424095,149000000,64459316,Flushed Away,Hugh Jackman|Kate Winslet|Ian McKellen|Jean Re...,David Bowers|Sam Fell,Someone's Going Down,"London high-society mouse, Roddy is flushed do...",85,Adventure|Animation|Comedy|Family,Aardman Animations|DreamWorks Animation,10/22/2006,6.0,2006
1037,tt1758830,35000000,88058786,This Is 40,Paul Rudd|Leslie Mann|Jason Segel|Maude Apatow...,Judd Apatow,The sort-of sequel to 'Knocked Up',"Pete and Debbie are both about to turn 40, the...",134,Comedy,Apatow Productions,12/20/2012,5.5,2012


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [4]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
from datetime import datetime
data['release_date'] = data['release_date'].apply(
                       lambda d: datetime.strptime(d, "%m/%d/%Y")
                       )

# Add new feature "profit" (profit = revenue - budget) 
data['profit'] = data['revenue'].sub(data['budget'])

#The dataset contains string values with inconvenient separator "|"
#So we need to split them into lists of strings for further counting:
# the names of actors:
data.cast = data.cast.str.split('|')
# directors: 
data.director = data.director.str.split('|')
# genres:
data.genres = data.genres.str.split('|')
# and also production_companies:
data.production_companies = data.production_companies.str.split('|')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   imdb_id               1889 non-null   object        
 1   budget                1889 non-null   int64         
 2   revenue               1889 non-null   int64         
 3   original_title        1889 non-null   object        
 4   cast                  1889 non-null   object        
 5   director              1889 non-null   object        
 6   tagline               1889 non-null   object        
 7   overview              1889 non-null   object        
 8   runtime               1889 non-null   int64         
 9   genres                1889 non-null   object        
 10  production_companies  1889 non-null   object        
 11  release_date          1889 non-null   datetime64[ns]
 12  vote_average          1889 non-null   float64       
 13  release_year      

In [6]:
data.head(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
0,tt0369610,150000000,1513528810,Jurassic World,"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan...",[Colin Trevorrow],The park is open.,Twenty-two years after the events of Jurassic ...,124,"[Action, Adventure, Science Fiction, Thriller]","[Universal Studios, Amblin Entertainment, Lege...",2015-06-09,6.5,2015,1363528810
1,tt1392190,150000000,378436354,Mad Max: Fury Road,"[Tom Hardy, Charlize Theron, Hugh Keays-Byrne,...",[George Miller],What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,"[Action, Adventure, Science Fiction, Thriller]","[Village Roadshow Pictures, Kennedy Miller Pro...",2015-05-13,7.1,2015,228436354
2,tt2908446,110000000,295238201,Insurgent,"[Shailene Woodley, Theo James, Kate Winslet, A...",[Robert Schwentke],One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,"[Adventure, Science Fiction, Thriller]","[Summit Entertainment, Mandeville Films, Red W...",2015-03-18,6.3,2015,185238201
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,"[Harrison Ford, Mark Hamill, Carrie Fisher, Ad...",[J.J. Abrams],Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,"[Action, Adventure, Science Fiction, Fantasy]","[Lucasfilm, Truenorth Productions, Bad Robot]",2015-12-15,7.5,2015,1868178225
4,tt2820852,190000000,1506249360,Furious 7,"[Vin Diesel, Paul Walker, Jason Statham, Miche...",[James Wan],Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,"[Action, Crime, Thriller]","[Universal Pictures, Original Film, Media Righ...",2015-04-01,7.3,2015,1316249360


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [7]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
# answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# +
# если ответили верно, можете добавить комментарий со значком "+"

In [8]:
# тут пишем ваш код для решения данного вопроса:
data[data.budget == data.budget.max()][['original_title', 'imdb_id']]

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


ВАРИАНТ 2

In [9]:
# можно добавлять разные варианты решения
data[data.budget == data.budget.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,"[Johnny Depp, PenÃ©lope Cruz, Geoffrey Rush, I...",[Rob Marshall],Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,"[Adventure, Action, Fantasy]","[Walt Disney Pictures, Jerry Bruckheimer Films...",2011-05-11,6.3,2011,641683000


# 2. Какой из фильмов самый длительный (в минутах)?

In [10]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)'
# +

In [11]:
data[data.runtime == data.runtime.max()][['original_title', 'imdb_id']]

,original_title,imdb_id
1157,Gods and Generals,tt0279111


# 3. Какой из фильмов самый короткий (в минутах)?





In [12]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'
# +

In [13]:
data[data.runtime == data.runtime.min()][['original_title', 'imdb_id']]

,original_title,imdb_id
768,Winnie the Pooh,tt1449283


# 4. Какова средняя длительность фильмов?


In [14]:
answers['4'] = '2. 110'
# +

In [15]:
round(data['runtime'].mean())

110

ВАРИАНТ 2

In [16]:
round(data.describe()['runtime']['mean'])

110.0

# 5. Каково медианное значение длительности фильмов? 

In [17]:
answers['5'] = '1. 107'
# +

In [18]:
int(data['runtime'].median())

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [19]:
answers['6'] = '5. Avatar (tt0499549)'
# +

In [20]:
# лучше код получения столбца profit вынести в Предобработку что в начале
data[data['profit'] == data['profit'].max()][['original_title', 'imdb_id']]

,original_title,imdb_id
239,Avatar,tt0499549


# 7. Какой фильм самый убыточный? 

In [21]:
answers['7'] = '5. The Lone Ranger (tt1210819)'
# +

In [22]:
data[data['profit'] == data['profit'].min()][['original_title', 'imdb_id']]

,original_title,imdb_id
1245,The Lone Ranger,tt1210819


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [23]:
answers['8'] = '1. 1478'
# +

In [24]:
len(data[data['profit'] > 0]['profit'])

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [25]:
answers['9'] = '4. The Dark Knight (tt0468569)'
# +

In [26]:
data_cash = data[data['release_year'] == 2008]
data_cash[data_cash['revenue'] == data_cash['revenue'].max()][
                                  ['original_title', 'imdb_id']
                                  ]

,original_title,imdb_id
599,The Dark Knight,tt0468569


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [27]:
answers['10'] = '5. The Lone Ranger (tt1210819)'
# +

In [28]:
data_loss = data[(data['release_year'] >= 2012)&(data['release_year'] <= 2014)]
data_loss[data_loss['profit'] == data_loss['profit'].min()][
                                 ['original_title', 'imdb_id']
                                 ]

,original_title,imdb_id
1245,The Lone Ranger,tt1210819


# 11. Какого жанра фильмов больше всего?

In [29]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

In [30]:
answers['11'] = '3. Drama'
# +

In [31]:
data_genres = data.copy()
splitted_genres = data_genres.explode('genres')
genres_count = splitted_genres.groupby(['genres'])['genres'].count()
genres_count.sort_values(ascending=False).index[0]

'Drama'

ВАРИАНТ 2

In [32]:
genres_list = []

for i in data.genres:
    genres_list.extend(i)

genres_count = Counter(genres_list)
genres_count.most_common(1)

[('Drama', 782)]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [33]:
answers['12'] = '1. Drama'
# +

In [34]:
genres_list = []

for i in data[data['profit'] > 0].genres:
    genres_list.extend(i)

profit_genre = Counter(genres_list)
profit_genre.most_common(1)

[('Drama', 560)]

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [35]:
answers['13'] = '5. Peter Jackson'
# +

In [36]:
df = data.copy()
dirs_cut = df.explode('director')
grouped_directors = dirs_cut.groupby(['director'])['profit'].sum()
grouped_directors.sort_values(ascending=False).index[0]

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [37]:
answers['14'] = '3. Robert Rodriguez'
# +

In [38]:
df = data.copy()
df_genres = df.explode('genres')
df_action = df_genres[df_genres.genres == 'Action']
df_director = df_action.explode('director')
grouped_directors = df_director.groupby(['director'])['genres'].count()
grouped_directors.sort_values(ascending=False).index[0]

'Robert Rodriguez'

ВАРИАНТ 2

In [39]:
df = data.copy()
df_genres = df.explode('genres')
df_director = df_genres.explode('director')
df2 = df_director.pivot_table(columns = 'genres', index = 'director',
                             values = 'original_title', aggfunc = 'count')
df2.loc[df2['Action'] == df2['Action'].max()].index[0]

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [40]:
answers['15'] = '3. Chris Hemsworth'
# +

In [41]:
df = data.copy()
data_12 = df[df.release_year == 2012]
df_actors = data_12.explode('cast')
actors_revenue = df_actors.groupby(['cast'])['revenue'].sum()
actors_revenue.sort_values(ascending=False).index[0]

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [42]:
answers['16'] = '3. Matt Damon'
# +

In [43]:
df = data.copy()
above_mean = df[df.budget > df.budget.mean()]
df_actors = above_mean.explode('cast')
actors_movies = df_actors.groupby(['cast'])['original_title'].count()
actors_movies.sort_values(ascending=False).index[0]

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [44]:
answers['17'] = '2. Action'
# +

In [45]:
df = data.copy()
df_actors = df.explode('cast')
df_nicolas = df_actors[df_actors.cast == 'Nicolas Cage']
df_genre = df_nicolas.explode('genres')
genre_count = df_genre.groupby(['genres'])['genres'].count()
genre_count.sort_values(ascending=False).index[0]

'Action'

ВАРИАНТ 2

In [46]:
df = data.copy()
df_actors = df.explode('cast')
df_nicolas = df_actors[df_actors.cast == 'Nicolas Cage']
df_genre = df_nicolas.explode('genres')
nicolas_count = Counter(df_genre.genres)
nicolas_count.most_common(1)

[('Action', 17)]

# 18. Самый убыточный фильм от Paramount Pictures

In [47]:
answers['18'] = '1. K-19: The Widowmaker'
# +

In [48]:
df = data.copy()
df_companies = df.explode('production_companies')
df_paramount = df_companies[
               df_companies.production_companies == 'Paramount Pictures'
               ]
df_looser = df_paramount[df_paramount.profit == df_paramount.profit.min()]
df_looser.original_title.values.tolist()[0]

'K-19: The Widowmaker'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [49]:
answers['19'] = '5. 2015'
# +

In [50]:
year_profit = data.groupby(['release_year'])['profit'].sum()
year_profit.sort_values(ascending=False).index[0]

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [51]:
answers['20'] = '1. 2014'
# +

In [52]:
df = data.copy()
df_companies = df.explode('production_companies')
df_warner_bros = df_companies[df_companies.production_companies
                             .str.contains('Warner Bros', na=False)
                             ]
grouped_year = df_warner_bros.groupby(['release_year'])['profit'].sum()
grouped_year.sort_values(ascending=False)[:1].index.tolist()

[2014]

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [53]:
answers['21'] = '4. Сентябрь'
# +

In [54]:
df = data.copy()
df['month'] = df['release_date'].dt.month
grouped_mov = df.groupby(['month'])['original_title'].count()
grouped_mov.sort_values(ascending=False).index[0]

9

ВАРИАНТ 2

In [55]:
#anoter variant of result displaying 
df = data.copy()
month_list = [
             'Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь',
             'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь'
             ]
df['month'] = df['release_date'].dt.month
grouped_index = df.groupby(['month'])['original_title'].count()
month_index = grouped_index.sort_values(ascending=False).index[0]
month_list[month_index-1]

'Сентябрь'

ВАРИАНТ 3

In [61]:
df = data.copy()
df['month'] = df['release_date'].dt.month



df_grouped = df.pivot_table(columns = 'release_year', index = 'month',
                     values = 'original_title', aggfunc = 'count', margins=True)
# we received the nice grouped table with whole view by_month/by_year
display(df_grouped)
#also we can extract the number of month with the max movie amount 
df3 = df_grouped['All'][:-1]
display(df3[df3 == df3.max()])

release_year,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,All
month,,,,,,,,,,,,,,,,,
1,2,4,4,3,5,9,5,5,7,12,9,10,7,11,9,8,110
2,7,7,10,8,9,8,8,10,11,8,8,12,7,8,8,6,135
3,9,7,11,6,7,8,13,11,5,12,11,15,10,10,12,9,156
4,10,7,8,11,15,6,9,12,10,9,10,13,10,7,5,7,149
5,7,5,7,8,8,11,9,8,9,7,8,11,11,11,12,8,140
6,8,10,7,6,11,13,9,12,9,10,12,5,12,10,4,9,147
7,8,11,7,8,8,10,8,8,10,8,9,12,3,12,9,11,142
8,5,11,11,8,6,9,12,13,13,11,10,9,12,10,13,8,161
9,5,8,9,9,10,21,18,14,12,22,14,21,19,19,9,17,227


month
9    227
Name: All, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [62]:
answers['22'] = '2. 450'
# +

In [63]:
df = data.copy()
df['month'] = df['release_date'].dt.month
df_month = df.groupby(['month'])['original_title'].count()
df_month[5:8].sum()

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [64]:
answers['23'] = '5. Peter Jackson'
# +

In [65]:
df = data.copy()
df['month'] = df['release_date'].dt.month
df_winter = df[(df.month == 1) | (df.month == 2) | (df.month == 12)]
df_director = df_winter.explode('director')
winter_product = df_director.groupby(['director'])['original_title'].count()
winter_product.sort_values(ascending=False)[:1].index.tolist()

['Peter Jackson']

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [66]:
answers['24'] = '5. Four By Two Productions'
# +

In [67]:
df = data.copy()
df_companies = df.explode('production_companies')
df_companies['title_lengh'] = df_companies['original_title'] \
                              .apply(lambda title:len(title))
companies = df_companies.groupby(['production_companies'])['title_lengh'].mean()
companies.sort_values(ascending=False)[:1].index.tolist()

['Four By Two Productions']

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [68]:
answers['25'] = '3. Midnight Picture Show'
# +

In [69]:
df = data.copy()
df_companies = df.explode('production_companies')
df_companies['overview_words'] = df_companies.overview.str.split().apply(len)
companies = df_companies.groupby(['production_companies']).overview_words.mean()
companies.sort_values(ascending=False)[:1].index.tolist()

['Midnight Picture Show']

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [70]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'
#+

In [71]:
df = data.copy()
one_percent = int(len(df.original_title)/100)
movies = df.groupby(['original_title'])['vote_average'].mean()
movies.sort_values(ascending=False)[:(one_percent+1)].index.tolist()

['The Dark Knight',
 'The Imitation Game',
 'Room',
 'Interstellar',
 'Inside Out',
 'The Grand Budapest Hotel',
 'The Pianist',
 'Guardians of the Galaxy',
 'Memento',
 'The Wolf of Wall Street',
 'Inception',
 'Gone Girl',
 '12 Years a Slave',
 'The Lord of the Rings: The Return of the King',
 'Prisoners',
 'The Fault in Our Stars',
 'Eternal Sunshine of the Spotless Mind',
 'The Prestige',
 'The Lord of the Rings: The Fellowship of the Ring']

ВАРИАНТ 2

In [72]:
df = data.copy()
one_percent = int(df.original_title.nunique()/100)
movies = df.groupby(['original_title'])['vote_average'].mean()
movies.sort_values(ascending=False)[:(one_percent+1)].index.tolist()

['The Dark Knight',
 'The Imitation Game',
 'Room',
 'Interstellar',
 'Inside Out',
 'The Grand Budapest Hotel',
 'The Pianist',
 'Guardians of the Galaxy',
 'Memento',
 'The Wolf of Wall Street',
 'Inception',
 'Gone Girl',
 '12 Years a Slave',
 'The Lord of the Rings: The Return of the King',
 'Prisoners',
 'The Fault in Our Stars',
 'Eternal Sunshine of the Spotless Mind',
 'The Prestige',
 'The Lord of the Rings: The Fellowship of the Ring']

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [73]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'
#+

In [74]:
df = data.copy()
cast_combi = [] 
df.cast.apply(lambda cast:cast_combi.extend(list(combinations(cast, 2)))) 
actor_s_pairs = pd.Series(cast_combi).value_counts()
display(actor_s_pairs)

(Daniel Radcliffe, Emma Watson)        8
(Daniel Radcliffe, Rupert Grint)       8
(Rupert Grint, Emma Watson)            7
(Johnny Depp, Helena Bonham Carter)    6
(Ben Stiller, Owen Wilson)             6
                                      ..
(Rick Malambri, Adam G. Sevani)        1
(Dakota Blue Richards, Sam Elliott)    1
(Joel Courtney, Kyle Chandler)         1
(Laurence Fishburne, Maria Bello)      1
(Richard Gere, Stanley Tucci)          1
Length: 18121, dtype: int64

ВАРИАНТ 2

In [75]:
df = data.copy()
cast_comb = [] 
for cast in df.cast:
    cast_comb.extend(list(combinations(cast, 2)))
actors_pairs = pd.Series(cast_comb).value_counts()
display(actors_pairs[:2])

(Daniel Radcliffe, Emma Watson)     8
(Daniel Radcliffe, Rupert Grint)    8
dtype: int64

# Submission

In [76]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [77]:
# и убедиться что ни чего не пропустил)
len(answers)

27